In [57]:
import numpy as np
import cv2
import base64
import requests
import os
import pickle

---------------------------------
Model Convert Step

In [58]:
#url_hoggen = 'http://192.168.1.5:8080/api/genhog'
url_hoggen = 'http://127.0.0.1:8000/api/genhog'
flowers_path = f"flower_photos/"
written_path = f'flower_models/'

In [59]:
def saveModel(obj, name):
    model = f'{written_path}{name}.pk'
    pickle.dump(obj, open(model, 'wb'))
    print(f'saved {model} successfully.')

def loadModel(name):
    return pickle.load(open(f'{written_path}{name}', 'rb'))

In [60]:
def getHOGData(img):
    # convert image to buffer
    v, buffer = cv2.imencode('.jpg', img)
    # convert buffer of image to base64
    img_str = "img,"+str.split(str(base64.b64encode(buffer)), "'")[1]

    req = requests.get(url_hoggen, json={"img":img_str})
    return req.json()['data']

type_dict = {
}

In [61]:
#convert cars dataset into HOG Data & save file
def convertAllAndSave(train_test=True, returned=False):
  hog_list = []
  type_list = []

  dest_path = 'train' if train_test else 'test'
  
  print(f"Converting {dest_path} plants into HOG Model...")
  i = 0
  for subFolder in os.listdir(flowers_path+dest_path):
      print(f'{i}. {subFolder}')
      type_dict[subFolder] = i
      try:
          subFolderRelPath = os.path.join(flowers_path+dest_path,subFolder)
          for file in os.listdir(subFolderRelPath):
            img = cv2.imread(subFolderRelPath+"/"+file)
            hog_list.append(getHOGData(img))
            type_list.append(type_dict[subFolder])
          
          i = i + 1
      except:
          print("error in dir: " + subFolder)

  print(f'\n{len(hog_list)} Model loaded complete!')
  saveModel(hog_list, ('' if train_test else 'test_') + 'flowerHog')
  saveModel(type_list, ('' if train_test else 'test_') + 'floweType')

  if returned:
    return {"hog": hog_list, "type": type_list}
  else:
    return None

Convert Model 

In [62]:
convertAllAndSave(True)

Converting train plants into HOG Model...
0. daisy
1. dandelion
2. roses
3. sunflowers
4. tulips

3176 Model loaded complete!
saved flower_models/flowerHog.pk successfully.
saved flower_models/floweType.pk successfully.


---------------------------------
Generate Model Step

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [64]:
hog_data = loadModel('flowerHog.pk')
type_data = loadModel('floweType.pk')

X_train = np.array(hog_data)
Y_train = np.array(type_data)


In [65]:
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)

DecisionTreeClassifier()

----------------------
test session

In [66]:
hog_data = convertAllAndSave(False, True)
test_hog_list = hog_data['hog']
test_type_list = hog_data['type']

Converting test plants into HOG Model...
0. daisy
1. dandelion
2. roses
3. sunflowers
4. tulips

3371 Model loaded complete!
saved flower_models/test_flowerHog.pk successfully.
saved flower_models/test_floweType.pk successfully.


In [30]:
hogs_np = np.array(test_hog_list)
type_np = np.array([[label] for label in test_type_list]) #make value inside into 2D array ([1,3,2,3] => [[1],[3],[2],[3]])
X_test = hogs_np
Y_test = type_np

In [31]:
X_train, X_test, y_train, y_test = train_test_split(hogs_np, type_np, test_size=0.2, random_state=42)
# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)*100
print("Accuracy:", accuracy)
print("Confusion Matrix:", accuracy)
confusion_matrix(y_test, y_pred)

Accuracy: 88.74074074074075
Confusion Matrix: 88.74074074074075


array([[126,   0,   0,   0,   0],
       [  0, 133,   0,   0,   0],
       [ 11,   8,  89,  10,  10],
       [  1,   2,   4, 128,   4],
       [  7,   8,   5,   6, 123]], dtype=int64)

In [33]:
saveModel(clf, 'flowersModel')

saved flower_models/flowersModel.pk successfully.
